In [1]:
import pandas as pd
import librosa
import cv2
import numpy as np
from PIL import Image
from moviepy.editor import *
import subprocess
import os
import glob
from removebg import RemoveBg
from aip import AipSpeech
import pymongo
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.faker import Faker
from snapshot_selenium import snapshot
from pyecharts.render import make_snapshot

C:\Users\AERO\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\AERO\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\AERO\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# 读取制作内容数据

In [9]:
# username = "zgtNewsUser"
# password = "ciiczgtJZFP"
# client = pymongo.MongoClient(host="localhost", port=27017, username=username, password=password)
# db = client['TCM_Datas']
# df = pd.DataFrame(list(db.TCM.find({})))

df = pd.read_excel("data/data.xlsx")
row = 8 # 这里设置数字
df = df.loc[row:row, :]
df.reset_index(drop=True, inplace=True)
pic_name = df["药材名称"][0].replace("?", "")
print("本次制作内容：", pic_name)

本次制作内容： 矮脚罗伞


In [10]:
# 重命名列信息
df = df.rename(columns={
    '药材名称':"CnName",
    '汉语拼音':"PinYin",
    '文件原文':"LaiYuan",
    '原拉丁植物动物矿物名':"LaDing",
    '药材基原':"YCJY",
    '性味归经':"YWGJ",
    '功能主治':"GNZZ",
    '地理分布':"QYFB",
    '省市':"SS",
    '英文名':"EnName",
    '药材别名':"YCBM",
    '化学成分':"HXCF", 
    '名家论述':"MJLS", 
    '拉丁文名':"LDWM", 
    '药理作用':"YLZY",
    '附方':"FF", 
    '现代临床研究':"XDLCYJ",
    '注意事项':"ZYSX", 
    '药用类别':"YYLB",
    '现代化研究':"XDHYJ"})
# df.drop(["_id"],inplace=True,axis=1)
# df.drop(["url"],inplace=True,axis=1)
# 清理掉无用的数据部分
df["QYFB"][0] = df["QYFB"][0].replace(" ","").replace("生态环境","").replace("资源分布","")
df = df.fillna("暂无数据")

# 加载配置内容

In [11]:
# 加载字体配置文件
font_path = './font/kaiti.ttf'
# 文件对应中文名称配置
text_dict = {
"CnName":"药材名称",
"PinYin":"汉语拼音",
"LaiYuan":"文件原文",
"LaDing":"原拉丁植物动物矿物名",
"YCJY":"药材基原",
"YWGJ":"性味归经",
"GNZZ":"功能主治",
"QYFB":"地理分布",
"SS":"省市",
"EnName":"英文名",
"YCBM":"药材别名",
"HXCF":"化学成分",
"MJLS":"名家论述",
"LDWM":"拉丁文名",
"YLZY":"药理作用",
"FF":"附方",
"XDLCYJ":"现代临床研究",
"ZYSX":"注意事项",
"YYLB":"药用类别",
"XDHYJ":"现代化研究",
}
# 文字处理对应方法
def clean_word(word):
    word_len = int(len(word) / 20)
    word_num = 0
    while word_num <= word_len:
        if word_num == 0:
            strs = word[:(word_num + 1) * 20] + "\n"
        else:
#             strs = word[:(word_num + 1) * 8] + "..."            
            strs = strs + word[word_num * 20:(word_num + 1) * 20] + "\n"
        word_num = word_num + 1
    return word_len,strs

In [12]:
def clean_word_2(word):
    word_len = int(len(word) / 10)
    word_num = 0
    while word_num <= word_len:
        if word_num == 0:
            strs = word[:(word_num + 1) * 10] + "\n"
        else:
#             strs = word[:(word_num + 1) * 8] + "..."            
            strs = strs + word[word_num * 10:(word_num + 1) * 10] + "\n"
        word_num = word_num + 1
    return word_len,strs

# 制作前删除合成语音的文件，否则无法继续

In [13]:
path = "./material_video/" + pic_name
for infile in glob.glob(os.path.join(path, '*.mp4')):
    os.remove(infile)
path = "./material_everypart/" + pic_name
for infile in glob.glob(os.path.join(path, '*.mp4')):
    os.remove(infile)
path = "./material_mp3/" + pic_name
for infile in glob.glob(os.path.join(path, '*.mp3')):
    os.remove(infile)

# 药材图片进行抠图生成素材

In [14]:
# 图片抠图处理
rmbg = RemoveBg("ypWN2SE5p57qKMk6jtaaSXRq", "data/error.log")  # 把你的`API Key`填进去
# 判断如果没有该数据的文件夹就创建
dirs = 'material_jpg/'+ pic_name
if not os.path.exists(dirs):
    os.makedirs(dirs)
# 判断视频图片是否存在，若存在则跳过
png_name = "./material_jpg/" + pic_name + "/" + pic_name + ".jpg_no_bg.png"
if not os.path.exists(png_name):
    rmbg.remove_background_from_img_file("./material_jpg/" + pic_name + "/" + pic_name + ".jpg")
else:
    print("该图片的内容抠图已经存在")

该图片的内容抠图已经存在


# 合成封面图片

In [15]:
# 加载背景图片
base_img = Image.open('material_jpg/base/base.jpg')
# 转换图片色到分别表示RGBA的值
target = Image.new('RGBA', base_img.size, (0, 0, 0, 0))
# 选择png图片显示的区域
box = (800, 400, 1260, 690)  
# 加载PNG图片
region = Image.open('material_jpg/' + pic_name + '/' + pic_name + '.jpg_no_bg.png')
# 确保图片是RGBA格式，大小和box区域一样
region = region.convert("RGBA")
region = region.resize((box[2] - box[0], box[3] - box[1]))
# 将素材图片合成道底板图上
target.paste(region,box)
# 将背景图上假如生成号的素材地板透明背景图
base_img.paste(target,(0,0),target) #第一个参数表示需要粘贴的图像，中间的是坐标，最后是一个是mask图片，用于指定透明区域，将底图显示出来。
# base_img.show()
base_img.save('material_jpg/' + pic_name + '/result.jpg')  # 保存图片

# 读取文字转语音

In [16]:
# 加载百度AIP账号
APP_ID = '22577460'
API_KEY = 'Q7aI5ALdYHbHzDGestq4trF8'
SECRET_KEY = 'h850Sj1g3eoLKDEwNAj6yVNyGx5GbXht'
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)
# 把文字转换成语音 将生成的音频文件保存到material_mp3下
def ChangeWordsToMp3(dataframe):
    # 判断如果没有该数据的文件夹就创建
    dirs = 'material_mp3/'+ dataframe["CnName"][0]
    if not os.path.exists(dirs):
        os.makedirs(dirs)
    # 遍历该条df数据每列文字进行语音MP3转换
    for col in dataframe.columns:
        try:
            result  = client.synthesis(text_dict[col] + dataframe[col][0], 'zh', 1, {'vol': 12,'spd':6,'per':0})
            if not isinstance(result, dict):
                with open('material_mp3/' + dataframe["CnName"][0] + "/" + col + '.mp3', 'wb') as f:
                    f.write(result)
        except:
            pass
# 执行函数生成文字对应MP3文件
ChangeWordsToMp3(df)

# 获取MP3的文件列表
def file_name(file_dir):
    list_ = [files for files in os.walk(file_dir)][0][2]
    filelist = [i for i in list_ if os.path.splitext(i)[1] == '.mp3']
    return filelist
filelist = file_name("./material_mp3/" + df["CnName"][0])
# 读取文字转语音的MP3 并计算时长
def get_mp3_duration(audio_path):
    duration = librosa.get_duration(filename=audio_path)
    return duration
time_name_dict = {}
time_num_all = 0  # 总音频的秒数
for i in filelist:
    time_num = get_mp3_duration("material_mp3/" + df["CnName"][0] + "/" + i)
    time_num_all = time_num_all + time_num
    time_name_dict[i] = time_num

拼接顺序制作
- "CnName":"药材名称"
- "LaiYuan":"文件原文"
- "YCBM":"药材别名",
- "YYLB":"药用类别",
    
    
- "PinYin":"汉语拼音",
- "LaDing":"原拉丁植物动物矿物名",
- "EnName":"英文名", 
- "LDWM":"拉丁文名",

    
- "YCJY":"药材基原",
- "YWGJ":"性味归经",
- "GNZZ":"功能主治",
- "ZYSX":"注意事项",
- "YLZY":"药理作用",


- "QYFB":"地理分布",
- "SS":"省市", # 自定义生成图表使用
    
    
- "MJLS":"名家论述",
- "HXCF":"化学成分",
- "XDLCYJ":"现代临床研究",
- "XDHYJ":"现代化研究",
    
    
- "FF":"附方",

In [17]:
df = df.fillna("暂无数据")

# 模板,如果字段为空则跳过

In [18]:
# 创建该内容的文件夹，用于保存算法合成的每个部分无声音的MP4视频 
dirs = 'material_everypart/'+ pic_name
if not os.path.exists(dirs):
    os.makedirs(dirs)

## 第一部分

In [20]:
# 选取第一部分的字段
word_list = ["CnName", "LaiYuan", "YCBM", "YYLB"]
data_list = []  # 选择有用的数据进行处理
f_write = open('material_mp3/' + pic_name + "/" + '1st.mp3', 'wb')  # 处理有用数据的MP3

for col in df[word_list].columns:
    if df[col][0] != "暂无数据":
        # 文本数据部分
        col_text = df[col]
        data_text = df[col][0]
        col_duration_time = time_name_dict[col + ".mp3"]
        one_dict = {col: {"text": data_text, "time": col_duration_time}}
        data_list.append(one_dict)

        # 语音数据部分
        f_read = open('material_mp3/' + pic_name + "/" + col + '.mp3', 'rb')
        f_write.write(f_read.read())
        f_read.close()
f_write.flush()
f_write.close()

start_time = 0
all_duration_time = sum([list(data_list[i].values())[0]["time"] for i in range(len(data_list))])

# 文字的坐标参数
title_y1 = 80
title_y2 = 140
title_y = 110

result_list = []

for i in range(len(data_list)):
    title = text_dict[list(data_list[i].keys())[0]]
    content = list(data_list[i].values())[0]["text"]
    duration_time = list(data_list[i].values())[0]["time"]

    # 判断内容过去分行
    if len(content) > 15:
        words_len, words_result = clean_word(content)
        content = "\n".join(words_result.split("\n")[:-1])
    else:
        words_len = 1

    ## 将标题和文字融入到视频中
    txt_title = (
        TextClip(title, font=font_path, fontsize=50, color='black', method='label', align='West')
            .set_position((230, title_y1))
            .set_duration(all_duration_time - start_time)
            .set_start(start_time)
    )
    txt_content = (
        TextClip(content, font=font_path, fontsize=40, color='black', method='label', align='West')
            .set_position((260, title_y2))
            .set_duration(all_duration_time - start_time)
            .set_start(start_time)
    )

    result_list.append(txt_title)
    result_list.append(txt_content)

    start_time = start_time + duration_time
    title_y1 = title_y1 + title_y + (words_len - 1) * 50
    title_y2 = title_y2 + title_y + (words_len - 1) * 50

# 添加该模块材料图片（药材图片）
Image = (
    ImageClip("./material_jpg/" + pic_name + "/" + pic_name + ".jpg_no_bg.png")
        .set_duration(start_time)  # 水印持续时间
        .resize(height=300)  # 水印的高度，会等比缩放
        .set_pos((600, 90))  # 水印的位置
)

# 加入水印
logo = (
    ImageClip("./material_jpg/base/logo.png")
        .set_duration(start_time)  # 水印持续时间
        .resize(height=50)  # 水印的高度，会等比缩放
        .set_pos(("right", "top"))  # 水印的位置
)

# 与背景进行合成
L = []
path = "material_base/1s.mp4"
video = VideoFileClip(path)
for i in range(int(all_duration_time) + 1):
    L.append(video)
final_clip = concatenate_videoclips(L).set_duration(all_duration_time).resize((1280, 720))
cvc = CompositeVideoClip([final_clip, Image, logo] + result_list, size=(1280, 720))
cvc.write_videofile("./material_everypart/" + pic_name + "/" + "1st.mp4", fps=60, remove_temp=False)

# 将对应的音频和视频进行合成
outfile_name = 'material_video/' + pic_name + '/' + '1st.mp4'
subprocess.call('ffmpeg -i ' + 'material_everypart/' + pic_name + '/' + '1st.mp4'
                + ' -i ' + 'material_mp3/' + pic_name + '/' + '1st.mp3' + ' -strict -2 -f mp4 '
                + outfile_name, shell=True)
print("第一部分内容处理完毕，如果发现黑屏请调整改部分的clean_word")

Moviepy - Building video ./material_everypart/矮脚罗伞/1st.mp4.
Moviepy - Writing video ./material_everypart/矮脚罗伞/1st.mp4



Moviepy - Done !
Moviepy - video ready ./material_everypart/矮脚罗伞/1st.mp4


0

## 第二部分

In [21]:
# 选取第二部分的字段
word_list = ["PinYin", "LaDing", "EnName", "LDWM"]

data_list = []  # 选择有用的数据进行处理
f_write = open('material_mp3/' + pic_name + "/" + '2nd.mp3', 'wb')  # 处理有用数据的MP3

for col in df[word_list].columns:
    if df[col][0] != "暂无数据":
        # 文本数据部分
        col_text = df[col]
        data_text = df[col][0]
        col_duration_time = time_name_dict[col + ".mp3"]
        one_dict = {col: {"text": data_text, "time": col_duration_time}}
        data_list.append(one_dict)

        # 语音数据部分
        f_read = open('material_mp3/' + pic_name + "/" + col + '.mp3', 'rb')
        f_write.write(f_read.read())
        f_read.close()
f_write.flush()
f_write.close()

start_time = 0
all_duration_time = sum([list(data_list[i].values())[0]["time"] for i in range(len(data_list))])

# 文字的坐标参数
title_y1 = 80
title_y2 = 140
title_y = 110

result_list = []

for i in range(len(data_list)):
    title = text_dict[list(data_list[i].keys())[0]]
    content = list(data_list[i].values())[0]["text"]
    duration_time = list(data_list[i].values())[0]["time"]

    ## 将标题和文字融入到视频中
    txt_title = (
        TextClip(title, font=font_path, fontsize=50, color='black', method='label', align='West')
            .set_position((230, title_y1))
            .set_duration(all_duration_time - start_time)
            .set_start(start_time)
    )
    txt_content = (
        TextClip(content, font=font_path, fontsize=40, color='black', method='label', align='West')
            .set_position((260, title_y2))
            .set_duration(all_duration_time - start_time)
            .set_start(start_time)
    )

    result_list.append(txt_title)
    result_list.append(txt_content)

    start_time = start_time + duration_time
    title_y1 = title_y1 + title_y
    title_y2 = title_y2 + title_y

# 添加该模块材料图片（药材图片）
Image = (
    ImageClip("./material_jpg/" + pic_name + "/" + pic_name + ".jpg_no_bg.png")
        .set_duration(start_time)  # 水印持续时间
        .resize(height=300)  # 水印的高度，会等比缩放
        .set_pos((600, 90))  # 水印的位置
)

# 加入水印
logo = (
    ImageClip("./material_jpg/base/logo.png")
        .set_duration(start_time)  # 水印持续时间
        .resize(height=50)  # 水印的高度，会等比缩放
        .set_pos(("right", "top"))  # 水印的位置
)

# 与背景进行合成
L = []
path = "material_base/1s.mp4"
video = VideoFileClip(path)
for i in range(int(all_duration_time) + 1):
    L.append(video)
final_clip = concatenate_videoclips(L).set_duration(all_duration_time).resize((1280, 720))
cvc = CompositeVideoClip([final_clip, Image, logo] + result_list, size=(1280, 720))
cvc.write_videofile("./material_everypart/" + pic_name + "/" + "2nd.mp4", fps=60, remove_temp=False)

# 将对应的音频和视频进行合成
outfile_name = 'material_video/' + pic_name + '/' + '2nd.mp4'
subprocess.call('ffmpeg -i ' + 'material_everypart/' + pic_name + '/' + '2nd.mp4'
                + ' -i ' + 'material_mp3/' + pic_name + '/' + '2nd.mp3' + ' -strict -2 -f mp4 '
                + outfile_name, shell=True)
print("第二部分内容处理完毕，如果发现黑屏请调整改部分的clean_word")

Moviepy - Building video ./material_everypart/矮脚罗伞/2nd.mp4.
Moviepy - Writing video ./material_everypart/矮脚罗伞/2nd.mp4



Moviepy - Done !
Moviepy - video ready ./material_everypart/矮脚罗伞/2nd.mp4


0

## 第三部分

In [22]:
# 选取第三部分的字段
word_list = ["YCJY","YWGJ","GNZZ","ZYSX","YLZY"]
data_list = [] # 选择有用的数据进行处理
f_write = open('material_mp3/' + pic_name + "/"  + '3rd.mp3' , 'wb') # 处理有用数据的MP3

for col in df[word_list].columns:
    if df[col][0] != "暂无数据":
        # 文本数据部分
        col_text = df[col]
        data_text = df[col][0]
        col_duration_time = time_name_dict[ col + ".mp3" ]
        one_dict = { col : { "text" : data_text , "time" : col_duration_time}}
        data_list.append(one_dict)
        
        # 语音数据部分
        f_read = open('material_mp3/' + pic_name + "/"  + col + '.mp3' , 'rb')
        f_write.write(f_read.read())
        f_read.close()
f_write.flush()
f_write.close()

start_time = 0
result_list = []
all_duration_time = sum([list(data_list[i].values())[0]["time"] for i in range(len(data_list))])

for i in range(len(data_list)):
    title = text_dict[list(data_list[i].keys())[0]]
    content = list(data_list[i].values())[0]["text"]
    duration_time = list(data_list[i].values())[0]["time"]


    # 获得文字长度
    words_len,words_result = clean_word(content) 
    # 构建切换的字幕方法
    str_list = words_result.split("\n")[:-1]
    allstrs = []  # 分组切分后的汉字放在这里
    every_list = []
    for i in range(len(str_list)):
        every_list.append((str_list[i]+"\n"))
        if i == 0:
            allstrs.append(every_list)
        if i%2 == 0 and i !=0:
            every_list = []
            allstrs.append(every_list)
        # 分多少页、和时长，由于切分音频是要 1|1|1|1 这么切分
    allstrs = [i for i in allstrs if i != [] ]  # 去除无用的空list避免报错
    allstrs = [i for i in allstrs if i != ['\n'] ]  # 去除无用的空list避免报错
    page_num , every_page_time = len(allstrs),  duration_time/len(allstrs)
#     print(len(allstrs),page_num , every_page_time)
#     print("起始时间：{}".format(start_time),"持续时间：{}".format(duration_time),"下段开始时间：{}".format(start_time + duration_time))
#     print("字段内容段落数：{}".format(page_num),"每段持续时间：{}".format(every_page_time))

    # 标题位置
    txt_title = (
        TextClip(title,font=font_path, fontsize=50, color='black', method='label',align = 'West')
            .set_position((210,320))
            .set_duration(duration_time)
            .set_start(start_time)
    )
    logo = (
        ImageClip("./material_jpg/base/logo.png")
            .set_duration(duration_time) # 水印持续时间
            .resize(height=50) # 水印的高度，会等比缩放
            .set_pos(("right","top")) # 水印的位置
    )
    Image = (
        ImageClip("./material_jpg/" + pic_name + "/" + pic_name + ".jpg_no_bg.png")
            .set_duration(all_duration_time) # 水印持续时间
            .resize(height=250) # 水印的高度，会等比缩放
            .set_pos(("center",90)) # 水印的位置
    )

    # 每个部分
    txt_list = []
    start_every = start_time

    for i in range(len(allstrs)):
        txt_every = (
            TextClip("".join(allstrs[i]), font=font_path, fontsize=40, color='black', method='label',align = 'West')
                .set_position((230,390))
                .set_duration(every_page_time)
                .set_start(start_every + every_page_time * i)
        )
    #         print(start_every + every_page_time * i)
        txt_list.append(txt_every)
    #         print("".join(allstrs[i]),start_time + every_page_time)
    start_time = start_time + duration_time

    result_list.append(txt_title)
    result_list = result_list + txt_list



L = []
path = "material_base/1s.mp4"
video = VideoFileClip(path)
for i in range(int(start_time)+1):
    L.append(video)
final_clip = concatenate_videoclips(L).set_duration(start_time).resize((1280,720))
cvc = CompositeVideoClip([final_clip,Image,logo] + result_list, size=(1280,720))
cvc.write_videofile("./material_everypart/" + pic_name + "/" + "3rd.mp4", fps=60, remove_temp=False,verbose=True)

# 将对应的音频和视频进行合成
outfile_name = 'material_video/' + pic_name + '/' + '3rd.mp4'
subprocess.call('ffmpeg -i ' + 'material_everypart/' + pic_name + '/' +'3rd.mp4'
                + ' -i ' + 'material_mp3/' + pic_name + '/' + '3rd.mp3' + ' -strict -2 -f mp4 '
                + outfile_name, shell=True)

Moviepy - Building video ./material_everypart/矮脚罗伞/3rd.mp4.
Moviepy - Writing video ./material_everypart/矮脚罗伞/3rd.mp4



Moviepy - Done !
Moviepy - video ready ./material_everypart/矮脚罗伞/3rd.mp4


0

## 第四部分

In [24]:
# 选取第四部分的字段
word_list = ["QYFB","SS"]
data_list = [] # 选择有用的数据进行处理
f_write = open('material_mp3/' + pic_name + "/"  + '4th.mp3' , 'wb') # 处理有用数据的MP3

for col in [df[word_list].columns[0]]:
    if df[col][0] != "暂无数据":
        # 文本数据部分
        col_text = df[col]
        data_text = df[col][0]
        col_duration_time = time_name_dict[ col + ".mp3" ]
        one_dict = { col : { "text" : data_text , "time" : col_duration_time,"area":df["SS"][0]}}
        data_list.append(one_dict)
        
        # 语音数据部分
        f_read = open('material_mp3/' + pic_name + "/"  + col + '.mp3' , 'rb')
        f_write.write(f_read.read())
        f_read.close()
f_write.flush()
f_write.close()

start_time = 0
result_list = []
all_duration_time = sum([list(data_list[i].values())[0]["time"] for i in range(len(data_list))])

for i in range(len(data_list)):
    title = text_dict[list(data_list[i].keys())[0]]
    content = list(data_list[i].values())[0]["text"]
    duration_time = list(data_list[i].values())[0]["time"]
    area = list(data_list[i].values())[0]["area"]

    # 获得文字长度
    words_len,words_result = clean_word(content) 
    # 构建切换的字幕方法
    str_list = words_result.split("\n")[:-1]
    allstrs = []  # 分组切分后的汉字放在这里
    every_list = []
    for i in range(len(str_list)):
        every_list.append((str_list[i]+"\n"))
        if i == 0:
            allstrs.append(every_list)
        if i%2 == 0 and i !=0:
            every_list = []
            allstrs.append(every_list)
        # 分多少页、和时长，由于切分音频是要 1|1|1|1 这么切分
    allstrs = [i for i in allstrs if i != [] ]  # 去除无用的空list避免报错
    allstrs = [i for i in allstrs if i != ['\n'] ]  # 去除无用的空list避免报错
    page_num , every_page_time = len(allstrs),  duration_time/len(allstrs)
#     print(len(allstrs),page_num , every_page_time)
#     print("起始时间：{}".format(start_time),"持续时间：{}".format(duration_time),"下段开始时间：{}".format(start_time + duration_time))
#     print("字段内容段落数：{}".format(page_num),"每段持续时间：{}".format(every_page_time))

    # 标题位置
    txt_title = (
        TextClip(title,font=font_path, fontsize=50, color='black', method='label',align = 'West')
            .set_position((210,320))
            .set_duration(duration_time)
            .set_start(start_time)
    )
    logo = (
        ImageClip("./material_jpg/base/logo.png")
            .set_duration(duration_time) # 水印持续时间
            .resize(height=50) # 水印的高度，会等比缩放
            .set_pos(("right","top")) # 水印的位置
    )
    # 根据地域信息生成png图
    # 制作省市显示图标数据bar
    n = 1
    list_data = []
    while n <= len(df["SS"][0]):
        list_data.append(1)
        n = n +1 
    name = pic_name + "的生长区域分布"
    # 制作背景图fuction    
    def every_map():
        c = (
        Map()
        .add(name, [list(z) for z in zip(area, list_data)], "china")
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(max_=1),
            )
        )
        return c
    make_snapshot(snapshot, every_map().render(), "./material_jpg/" + pic_name + "/area.png")
    
    # 将图片进行裁剪
    from PIL import Image
    img = Image.open("./material_jpg/" + pic_name + "/area.png")
    img_size = img.size
    h = img_size[1]  # 图片高度
    w = img_size[0]  # 图片宽度
    x = 0.2 * w
    y = 0
    w = 0.8 * w
    h = h
    # 开始截取
    region = img.crop((x, y, x + w, y + h))
    region.save("./material_jpg/" + pic_name + "/area_cut.png")
    
    Image = (
        ImageClip("./material_jpg/" + pic_name + "/area_cut.png")
            .set_duration(all_duration_time) # 水印持续时间
            .resize(height=500) # 水印的高度，会等比缩放
            .set_pos((600,90)) # 水印的位置
    )

    # 每个部分
    txt_list = []
    start_every = start_time

    for i in range(len(allstrs)):
        txt_every = (
            TextClip("".join(allstrs[i]), font=font_path, fontsize=40, color='black', method='label',align = 'West')
                .set_position((230,390))
                .set_duration(every_page_time)
                .set_start(start_every + every_page_time * i)
        )
    #         print(start_every + every_page_time * i)
        txt_list.append(txt_every)
    #         print("".join(allstrs[i]),start_time + every_page_time)
    start_time = start_time + duration_time

    result_list.append(txt_title)
    result_list = result_list + txt_list
    
L = []
path = "material_base/1s.mp4"
video = VideoFileClip(path)
for i in range(int(start_time)+1):
    L.append(video)
final_clip = concatenate_videoclips(L).set_duration(start_time).resize((1280,720))
cvc = CompositeVideoClip([final_clip,Image,logo] + result_list, size=(1280,720))
cvc.write_videofile("./material_everypart/" + pic_name + "/" + "4th.mp4", fps=60, remove_temp=False,verbose=True)

# 将对应的音频和视频进行合成
outfile_name = 'material_video/' + pic_name + '/' + '4th.mp4'
subprocess.call('ffmpeg -i ' + 'material_everypart/' + pic_name + '/' +'4th.mp4'
                + ' -i ' + 'material_mp3/' + pic_name + '/' + '4th.mp3' + ' -strict -2 -f mp4 '
                + outfile_name, shell=True)

Moviepy - Building video ./material_everypart/矮脚罗伞/4th.mp4.
Moviepy - Writing video ./material_everypart/矮脚罗伞/4th.mp4



Moviepy - Done !
Moviepy - video ready ./material_everypart/矮脚罗伞/4th.mp4


0

## 第五部分

In [78]:
# 选取第五部分的字段
word_list = ["MJLS","HXCF","XDLCYJ","XDHYJ"]
data_list = [] # 选择有用的数据进行处理
f_write = open('material_mp3/' + pic_name + "/"  + '5th.mp3' , 'wb') # 处理有用数据的MP3

for col in df[word_list].columns:
    if df[col][0] != "暂无数据":
        # 文本数据部分
        col_text = df[col]
        data_text = df[col][0]
        col_duration_time = time_name_dict[ col + ".mp3" ]
        one_dict = { col : { "text" : data_text , "time" : col_duration_time}}
        data_list.append(one_dict)
        
        # 语音数据部分
        f_read = open('material_mp3/' + pic_name + "/"  + col + '.mp3' , 'rb')
        f_write.write(f_read.read())
        f_read.close()
f_write.flush()
f_write.close()

start_time = 0
result_list = []
all_duration_time = sum([list(data_list[i].values())[0]["time"] for i in range(len(data_list))])

for i in range(len(data_list)):
    title = text_dict[list(data_list[i].keys())[0]]
    content = list(data_list[i].values())[0]["text"]
    duration_time = list(data_list[i].values())[0]["time"]


    # 获得文字长度
    words_len,words_result = clean_word(content) 
    # 构建切换的字幕方法
    str_list = words_result.split("\n")[:-1]
    allstrs = []  # 分组切分后的汉字放在这里
    every_list = []
    for i in range(len(str_list)):
        every_list.append((str_list[i]+"\n"))
        if i == 0:
            allstrs.append(every_list)
        if i%2 == 0 and i !=0:
            every_list = []
            allstrs.append(every_list)
        # 分多少页、和时长，由于切分音频是要 1|1|1|1 这么切分
    allstrs = [i for i in allstrs if i != [] ]  # 去除无用的空list避免报错
    allstrs = [i for i in allstrs if i != ['\n'] ]  # 去除无用的空list避免报错
    page_num , every_page_time = len(allstrs),  duration_time/len(allstrs)
#     print(len(allstrs),page_num , every_page_time)
#     print("起始时间：{}".format(start_time),"持续时间：{}".format(duration_time),"下段开始时间：{}".format(start_time + duration_time))
#     print("字段内容段落数：{}".format(page_num),"每段持续时间：{}".format(every_page_time))
    
        # 标题位置
    txt_title = (
        TextClip(title,font=font_path, fontsize=50, color='black', method='label',align = 'West')
            .set_position((210,320))
            .set_duration(duration_time)
            .set_start(start_time)
    )
    logo = (
        ImageClip("./material_jpg/base/logo.png")
            .set_duration(duration_time) # 水印持续时间
            .resize(height=50) # 水印的高度，会等比缩放
            .set_pos(("right","top")) # 水印的位置
    )
    Image = (
        ImageClip("./material_jpg/base/yanjiu.jpg")
            .set_duration(all_duration_time) # 水印持续时间
            .resize(height=250) # 水印的高度，会等比缩放
            .set_pos(("center",90)) # 水印的位置
    )

    # 每个部分
    txt_list = []
    start_every = start_time

    for i in range(len(allstrs)):
        txt_every = (
            TextClip("".join(allstrs[i]), font=font_path, fontsize=40, color='black', method='label',align = 'West')
                .set_position((230,390))
                .set_duration(every_page_time)
                .set_start(start_every + every_page_time * i)
        )
    #         print(start_every + every_page_time * i)
        txt_list.append(txt_every)
        print("".join(allstrs[i]),start_time + every_page_time)
    start_time = start_time + duration_time

    result_list.append(txt_title)
    result_list = result_list + txt_list
    
L = []
path = "material_base/1s.mp4"
video = VideoFileClip(path)
for i in range(int(start_time)+1):
    L.append(video)
final_clip = concatenate_videoclips(L).set_duration(start_time).resize((1280,720))
cvc = CompositeVideoClip([final_clip,Image,logo] + result_list, size=(1280,720))
cvc.write_videofile("./material_everypart/" + pic_name + "/" + "5th.mp4", fps=60, remove_temp=False,verbose=True)

# 将对应的音频和视频进行合成
outfile_name = 'material_video/' + pic_name + '/' + '5th.mp4'
subprocess.call('ffmpeg -i ' + 'material_everypart/' + pic_name + '/' +'5th.mp4'
                + ' -i ' + 'material_mp3/' + pic_name + '/' + '5th.mp3' + ' -strict -2 -f mp4 '
                + outfile_name, shell=True)

《植物名实图考》：治除打损伤，风痛。《陆
川本草》：根：接骨，活血，止痛、消肿。治
损伤肿痛。《四川中药志》：全草：治咳嗽吐
 10.55
血，气痛及寒湿腰痛。根：治冷气腹痛。《福
建中草药》：祛风除湿，通经活络。
 10.55
Moviepy - Building video ./material_everypart/矮脚罗伞/5th.mp4.
Moviepy - Writing video ./material_everypart/矮脚罗伞/5th.mp4



Moviepy - Done !
Moviepy - video ready ./material_everypart/矮脚罗伞/5th.mp4


0

## 第六部分

In [19]:
# 选取第五部分的字段
word_list = ["FF"]
data_list = [] # 选择有用的数据进行处理
f_write = open('material_mp3/' + pic_name + "/"  + '6th.mp3' , 'wb') # 处理有用数据的MP3

for col in df[word_list].columns:
    if df[col][0] != "暂无数据":
        # 文本数据部分
        col_text = df[col]
        data_text = df[col][0]
        col_duration_time = time_name_dict[ col + ".mp3" ]
        one_dict = { col : { "text" : data_text , "time" : col_duration_time}}
        data_list.append(one_dict)
        
        # 语音数据部分
        f_read = open('material_mp3/' + pic_name + "/"  + col + '.mp3' , 'rb')
        f_write.write(f_read.read())
        f_read.close()
f_write.flush()
f_write.close()

start_time = 0
result_list = []
all_duration_time = sum([list(data_list[i].values())[0]["time"] for i in range(len(data_list))])

for i in range(len(data_list)):
    title = text_dict[list(data_list[i].keys())[0]]
    content = list(data_list[i].values())[0]["text"]
    duration_time = list(data_list[i].values())[0]["time"]


    # 获得文字长度
    words_len,words_result = clean_word(content) 
    # 构建切换的字幕方法
    str_list = words_result.split("\n")[:-1]
    allstrs = []  # 分组切分后的汉字放在这里
    every_list = []
    for i in range(len(str_list)):
        every_list.append((str_list[i]+"\n"))
        if i == 0:
            allstrs.append(every_list)
        if i%2 == 0 and i !=0:
            every_list = []
            allstrs.append(every_list)
        # 分多少页、和时长，由于切分音频是要 1|1|1|1 这么切分
    allstrs = [i for i in allstrs if i != [] ]  # 去除无用的空list避免报错
    allstrs = [i for i in allstrs if i != ['\n'] ]  # 去除无用的空list避免报错
    page_num , every_page_time = len(allstrs),  duration_time/len(allstrs)
#     print(len(allstrs),page_num , every_page_time)
#     print("起始时间：{}".format(start_time),"持续时间：{}".format(duration_time),"下段开始时间：{}".format(start_time + duration_time))
#     print("字段内容段落数：{}".format(page_num),"每段持续时间：{}".format(every_page_time))

    # 标题位置
    txt_title = (
        TextClip(title,font=font_path, fontsize=50, color='black', method='label',align = 'West')
            .set_position((210,320))
            .set_duration(duration_time)
            .set_start(start_time)
    )
    logo = (
        ImageClip("./material_jpg/base/logo.png")
            .set_duration(duration_time) # 水印持续时间
            .resize(height=50) # 水印的高度，会等比缩放
            .set_pos(("right","top")) # 水印的位置
    )
    Image = (
        ImageClip("./material_jpg/base/fufang.jpg")
            .set_duration(all_duration_time) # 水印持续时间
            .resize(height=250) # 水印的高度，会等比缩放
            .set_pos(("center",90)) # 水印的位置
    )

    # 每个部分
    txt_list = []
    start_every = start_time

    for i in range(len(allstrs)):
        txt_every = (
            TextClip("".join(allstrs[i]), font=font_path, fontsize=40, color='black', method='label',align = 'West')
                .set_position((230,390))
                .set_duration(every_page_time)
                .set_start(start_every + every_page_time * i)
        )
    #         print(start_every + every_page_time * i)
        txt_list.append(txt_every)
    #         print("".join(allstrs[i]),start_time + every_page_time)
    start_time = start_time + duration_time

    result_list.append(txt_title)
    result_list = result_list + txt_list



L = []
path = "material_base/1s.mp4"
video = VideoFileClip(path)
for i in range(int(start_time)+1):
    L.append(video)
final_clip = concatenate_videoclips(L).set_duration(start_time).resize((1280,720))
cvc = CompositeVideoClip([final_clip,Image,logo] + result_list, size=(1280,720))
cvc.write_videofile("./material_everypart/" + pic_name + "/" + "6th.mp4", fps=60, remove_temp=False,verbose=True)

# 将对应的音频和视频进行合成
outfile_name = 'material_video/' + pic_name + '/' + '6th.mp4'
subprocess.call('ffmpeg -i ' + 'material_everypart/' + pic_name + '/' +'6th.mp4'
                + ' -i ' + 'material_mp3/' + pic_name + '/' + '6th.mp3' + ' -strict -2 -f mp4 '
                + outfile_name, shell=True)

Moviepy - Building video ./material_everypart/阿尔泰扭藿香/6th.mp4.
Moviepy - Writing video ./material_everypart/阿尔泰扭藿香/6th.mp4



Moviepy - Done !
Moviepy - video ready ./material_everypart/阿尔泰扭藿香/6th.mp4


1

# 制作封面

In [20]:
# 将封面生成1秒的视频
from PIL import Image

fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
size = (1280, 720)
vw = cv2.VideoWriter("./material_everypart/" + pic_name + "/" + "cover.mp4", fourcc=fourcc, fps=10, frameSize=size)
f_read = cv2.imdecode(np.fromfile("./material_jpg/"+ pic_name + "/result.jpg",dtype=np.uint8),cv2.IMREAD_COLOR)
f_img = Image.fromarray(f_read)
f_rs = f_img.resize([1280, 720], resample=Image.NONE)
f_out = np.array(f_rs)
for i in range(10):
    vw.write(f_out)
vw.release()

name = pic_name

Txt = (
    TextClip(name, font='./font/kaiti.ttf', fontsize=80, color='black', method='label')
        .set_position(("center",420))
        .set_duration(1) # 水印持续时间
)

path = "material_everypart/" + pic_name + "/" + "cover.mp4"
video = VideoFileClip(path).resize((1280, 720))

cvc = CompositeVideoClip([video,Txt], size=(1280, 720))
cvc.write_videofile("./material_video/" + pic_name + "/" + "cover.mp4", fps=60, remove_temp=False,verbose=False)

Moviepy - Building video ./material_video/阿尔泰扭藿香/cover.mp4.
Moviepy - Writing video ./material_video/阿尔泰扭藿香/cover.mp4



Moviepy - Done !
Moviepy - video ready ./material_video/阿尔泰扭藿香/cover.mp4


# 按照顺序将视频进行拼接

In [22]:
file_dir = "material_video/" + pic_name +"/"
video_list = [files for files in os.walk(file_dir)][0][2]

# 素材路径
path_material = "material_video/" + pic_name + "/"
# 切换过场路径
path_chanege = "material_base/change.mp4"
# 封面文件路径
path_cover = "./material_video/" + pic_name + "/" + "cover.mp4"
# 结尾文件路径
path_end = "material_base/end.mp4"

L = []
video = VideoFileClip(path_cover).resize((1280,720))
L.append(video)
if "1st.mp4" in video_list:
    video = VideoFileClip(path_material + "1st.mp4").resize((1280,720)).fadein(2,(1,1,1))
    L.append(video)
if "2nd.mp4" in video_list:
    video = VideoFileClip(path_chanege).resize((1280,720))
    L.append(video)
    video = VideoFileClip(path_material + "2nd.mp4").resize((1280,720))
    video = video.set_duration(video.duration-0.5)
    L.append(video)
if "3rd.mp4" in video_list:
    video = VideoFileClip(path_chanege).resize((1280,720))
    L.append(video)
    video = VideoFileClip(path_material + "3rd.mp4").resize((1280,720))
    video = video.set_duration(video.duration-0.5)
    L.append(video)
if "4th.mp4" in video_list:
    video = VideoFileClip(path_chanege).resize((1280,720))
    L.append(video)
    video = VideoFileClip(path_material + "4th.mp4").resize((1280,720))
    video = video.set_duration(video.duration-0.5)
    L.append(video)   
if "5th.mp4" in video_list:
    video = VideoFileClip(path_chanege).resize((1280,720))
    L.append(video)
    video = VideoFileClip(path_material + "5th.mp4").resize((1280,720))
    video = video.set_duration(video.duration-0.5)
    L.append(video) 
if "6th.mp4" in video_list:
    video = VideoFileClip(path_chanege).resize((1280,720))
    L.append(video)
    video = VideoFileClip(path_material + "6th.mp4").resize((1280,720))
    video = video.set_duration(video.duration-0.5)
    L.append(video)  
    
video = VideoFileClip(path_end).resize((1280,720)).fadein(2,(1,1,1))
L.append(video)

final_clip = concatenate_videoclips(L)

# 生成目标视频文件
final_clip.to_videofile(path_material + "result.mp4", fps=60, remove_temp=False)

Moviepy - Building video material_video/阿尔泰扭藿香/result.mp4.
MoviePy - Writing audio in resultTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video material_video/阿尔泰扭藿香/result.mp4



Moviepy - Done !
Moviepy - video ready material_video/阿尔泰扭藿香/result.mp4


# 将配乐和视频进行合成

In [23]:
inmp4 = 'material_video/' + pic_name + '/'+ 'result.mp4'
inmp3 = 'material_mp3/music.mp3'
outmp4 = 'material_result/' + pic_name + '.mp4'

cmd='ffmpeg -y -i '+ inmp4 +' -i '+ inmp3 +' -filter_complex \
"[0:a]volume=10dB[a0]; \
[1:a]volume=-10dB[a1]; \
[a0][a1]amix=inputs=2[a]" \
-map 0:v -map "[a]" -c:v copy -c:a aac -shortest '+ outmp4

p = subprocess.call(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [24]:
# 删除目录下生成的MP3文件
path = "./"
for infile in glob.glob(os.path.join(path, '*.mp3')):
    os.remove(infile)
for infile in glob.glob(os.path.join(path, '*.html')):
    os.remove(infile)